In [ ]:
import pandas as pd
import scanpy as sc

In [ ]:
# biogrid data
# entrez id to ensembl id
edge_list = pd.read_csv("../../data/BIOGRID/BIOGRID-ORGANISM-Homo_sapiens-4.4.218.tab3.txt", sep="\t", low_memory=False)
ensembl2entrez = pd.read_csv("../../data/Ensembl/Ensembl2Entrez.txt", sep="\t", header=None).dropna().copy()
ensembl2entrez[1] = ensembl2entrez[1].astype(int)
entrez2ensembl_dict = ensembl2entrez.set_index(1)[0].to_dict()
edge_list = edge_list[edge_list['Entrez Gene Interactor A'].apply(lambda x: str(x).isdigit())]
edge_list = edge_list[edge_list['Entrez Gene Interactor B'].apply(lambda x: str(x).isdigit())]
edge_list[0] = edge_list['Entrez Gene Interactor A'].astype(int).map(entrez2ensembl_dict)
edge_list[1] = edge_list['Entrez Gene Interactor B'].astype(int).map(entrez2ensembl_dict)
edge_list = edge_list[edge_list['Experimental System Type'] == "physical"].copy()

In [ ]:
edge_list = edge_list.loc[:, [0, 1]].dropna()

In [ ]:
# convert ensembl ID to gene name
data = sc.read_h5ad("/home/jg2447/slayman/perturb/data/perturb/K562_gwps_normalized_bulk_01.h5ad")
id2name = data.var['gene_name'].to_dict()
edge_list[0] = edge_list[0].map(id2name)
edge_list[1] = edge_list[1].map(id2name)
# keep only both ends existed in the full perturb-seq data
edge_list = edge_list.dropna()
edge_list = edge_list.loc[:, [0, 1]].copy()

In [ ]:
# map to data index
gene = pd.read_csv("../../result/data/genes_gene_perturb", header=None)
gene_dict = gene.reset_index().set_index(0)['index'].to_dict()
edge_list[0] = edge_list[0].map(gene_dict)
edge_list[1] = edge_list[1].map(gene_dict)

In [ ]:
# remove nodes in full perturb-seq data (~9000) and keep only significant one (~2319)
edge_list = edge_list.dropna().astype(int)

In [ ]:
edge_list.to_csv("../../result/network/BIOGRID.tsv", sep="\t", header=False, index=False)